# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
part1_data = "output_data/Data Frame.csv"
weather_df = pd.read_csv(part1_data)
weather_df = weather_df.drop(['Unnamed: 0'], axis = 1 )


weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,66.99,72,90,8.05,AU,1613043462
1,Sola,-13.8833,167.5500,82.08,78,17,11.86,VU,1613043462
2,Husavik,66.0449,-17.3389,32.00,74,0,11.50,IS,1613043462
3,Taicheng,38.7310,113.2280,40.01,63,89,2.80,CN,1613043463
4,Rikitea,-23.1203,-134.9692,78.10,79,90,11.68,PF,1613043463


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
#configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)


fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity=100,
                                 point_radius = 1) 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Ideal Temperature 60F - 90F
#Ideal Humidity 30% - 50%
#Ideal Cloudiness 0-7
#Ideal Wind Speed 0 - 15

ideal_weather = weather_df.loc[(weather_df["Max Temp"] >= 60) &
                               (weather_df["Max Temp"]<= 90) &
                               (weather_df["Wind Speed"] <= 15) &
                               (weather_df["Humidity"] <= 50) &
                               (weather_df["Cloudiness"] <= 7 ),:
                              ]
# ideal_weather.head()

if len(ideal_weather) > 10:
    ideal_weather = ideal_weather
    
ideal_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,Sakakah,29.9697,40.2064,71.60,26,0,5.75,SA,1613043465
97,Kédougou,12.5579,-12.1743,84.20,21,0,6.91,SN,1613043478
111,Kutum,14.2000,24.6667,81.12,16,0,6.04,SD,1613043480
125,Saint-Louis,16.3333,-15.0000,82.40,24,0,2.30,SN,1613043483
141,Maţāy,28.4190,30.7792,80.37,16,0,5.14,EG,1613043485


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df = pd.DataFrame(ideal_weather)
hotel_df["Hotel Name"] = ""
#hotel_df.head()

params = {"radius": 5000, "types": "lodging",
          "keyword": "Hotel", "key": g_key}

for index, row in hotel_df.iterrows():
    hotel_lat = row["Lat"]
    hotel_lng = row["Lng"]
    params["location"] = f"{hotel_lat},{hotel_lng}"
    
    hotel_data = requests.get(base_url, params = params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        print("Hotel found")
    except(IndexError, KeyError):
        print("Hotel not found")

hotel_df = hotel_df.dropna()
hotel_df.head()

Hotel found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel not found
Hotel found


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,Sakakah,29.9697,40.2064,71.60,26,0,5.75,SA,1613043465,Pearl Palace Hotel Suites
97,Kédougou,12.5579,-12.1743,84.20,21,0,6.91,SN,1613043478,Relais de Kédougou
111,Kutum,14.2000,24.6667,81.12,16,0,6.04,SD,1613043480,
125,Saint-Louis,16.3333,-15.0000,82.40,24,0,2.30,SN,1613043483,
141,Maţāy,28.4190,30.7792,80.37,16,0,5.14,EG,1613043485,


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [35]:
gmaps.configure(api_key = g_key)

location2 = hotel_df[["Lat", "Lng"]].astype(float)

hotel_df = hotel_df.dropna()
weight2 = hotel_df.astype(float)


fig2 = gmaps.figure()

hotel_layer = gmaps.heatmap_layer(location2, hotel_info)

fig.add_layer(hotel_layer)
fig2
# Add marker layer ontop of heat map
# Display figure

ValueError: could not convert string to float: 'Sakakah'